In [27]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import yfinance as yf
import pandas as pd
from collections import Counter

# OUR DATA 

In [28]:
symbol = '^NSEI'
period = '3d'
interval = "15m"
ticker = yf.Ticker(symbol)

df = ticker.history(period=period, interval=interval)
df.reset_index(drop=False, inplace=True)

In [29]:
df['Datetime'] = pd.to_datetime(df['Datetime'])

In [30]:
df['Date'] = df['Datetime'].dt.date
df['Time'] = df['Datetime'].dt.time
date_range = pd.DatetimeIndex(df['Date'])

In [31]:
df.drop(columns=["Volume","Dividends","Stock Splits"],inplace=True)

current = df.tail(1)

# DATE 

In [32]:
current_date = datetime.datetime.now().strftime('%Y-%m-%d')
print("current_date :",current_date)
date_tuples = [(date.year, date.month, date.day) for date in set(df['Datetime'].dt.date)]
date_tuples.sort()
formatted_dates = [f'{date[0]:04d}-{date[1]:02d}-{date[2]:02d}' for date in date_tuples]
previous_day_data = df[df['Datetime'].dt.date == pd.to_datetime('2023-09-01').date()]
previous_date= formatted_dates[-2]
print("previous_data :",previous_date)

current_date : 2023-09-07
previous_data : 2023-09-05


In [33]:
previous_day_data = df[df['Datetime'].dt.date <= pd.to_datetime(previous_date).date() ]

## PREVIOUS DAY HIGH LOW

In [34]:
max_high_row = previous_day_data[previous_day_data['High'] == previous_day_data['High'].max()]["High"]
min_low_row = previous_day_data[previous_day_data['Low'] == previous_day_data['Low'].min()]["Low"]
print("Max High:",max_high_row)
print("Max Low:",min_low_row)

Max High: 35    19586.900391
Name: High, dtype: float64
Max Low: 5    19433.150391
Name: Low, dtype: float64


## MAX AND MIN CANDLES CONFLICT

In [83]:
def count_m_values(row):
    n = 1
    m_counts = {}
    for m in range(int(row['Low']) - n, int(row['High']) + n + 1):
        m_counts[m] = m_counts.get(m, 0) + 1
    return pd.Series(m_counts)

m_counts_df = previous_day_data.apply(count_m_values, axis=1).fillna(0).astype(int)

total_counts = m_counts_df.sum()
max_candle_re = total_counts.reset_index().rename(columns={'index': 'm', 0: 'Count'}).sort_values(by='Count', ascending=False)
max_candle_re['m_diff'] = max_candle_re['m'].diff().abs()
filtered_max_candle_re = max_candle_re[(max_candle_re['m_diff'] >= 20)]
filtered_max_candle_re = filtered_max_candle_re.drop('m_diff', axis=1).head(5)
filtered_max_candle_re.values.tolist()

[[19577, 13], [19508, 12], [19557, 12], [19509, 12], [19569, 12]]

,m,Count,Status
155,19587,1,SUPPORT
1,19433,2,SUPPORT
153,19585,3,SUPPORT
12,19444,4,SUPPORT
90,19522,5,SUPPORT
51,19483,5,SUPPORT
96,19528,5,SUPPORT
55,19487,5,SUPPORT
97,19529,5,SUPPORT
152,19584,6,SUPPORT


In [78]:
int(current['Close'].values)

19618